In [1]:
import nibabel as nib
import numpy as np

# NIfTI 파일 로드
file_path = '/nas/research/03-Neural_decoding/3-bids/derivatives/normalize/sub-02/betas_session02.nii.gz'
img = nib.load(file_path)
data = img.get_fdata()

print("Shape of the NIfTI image:", data.shape)

# t=1일 때의 볼륨
volume_t1 = data
# 값이 1인 복셀 위치 찾기
indices = np.argwhere(volume_t1 == 1)
print("t=1에서 bold 값이 1인 복셀 개수:", len(indices))



Shape of the NIfTI image: (82, 106, 84, 750)
t=1에서 bold 값이 1인 복셀 개수: 0


In [3]:
import nibabel as nib

# NIfTI 파일 경로
file_path = '/nas/research/03-Neural_decoding/3-bids/derivatives/b4_roi/sub-02/nsdgeneral.nii.gz'  # 또는 .nii

# 파일 로드
img = nib.load(file_path)

# 데이터 배열 추출
data = img.get_fdata()

# shape 출력
print("Shape of the NIfTI image:", data.shape)


Shape of the NIfTI image: (82, 106, 84)


In [ ]:
import nibabel as nib
import numpy as np

# 파일 경로
beta_path = '/nas/research/03-Neural_decoding/3-bids/derivatives/normalize/sub-01/betas_session03.nii.gz'

# 데이터 로드
beta_img = nib.load(beta_path)
beta_data = beta_img.get_fdata()  # shape: (x, y, z, t)

# t=1 (두 번째 timepoint)의 3D 볼륨
volume_t1 = beta_data[..., 1]  # shape: (x, y, z)

# 값이 0이 아닌 (x, y, z) 좌표의 개수 세기
non_zero_voxel_count = np.count_nonzero(volume_t1)

print(f"✅ t=1에서 intensity가 0이 아닌 복셀 수: {non_zero_voxel_count}")


In [5]:
import nibabel as nib
import numpy as np

# 파일 경로
beta_path = '/nas/research/03-Neural_decoding/3-bids/derivatives/normalize/sub-01/betas_session03.nii.gz'

# NIfTI 파일 로드
img = nib.load(beta_path)
data = img.get_fdata()  # shape: (x, y, z, t)

# 원하는 위치 지정
x, y, z, t = 35, 26, 40, 1  # 예: ITK-SNAP에서 본 좌표

# intensity 출력
value = data[x, y, z, t]
print(f"📍 Intensity at (x={x}, y={y}, z={z}, t={t}): {value}")


📍 Intensity at (x=35, y=26, z=40, t=1): 0.0


In [3]:
# beta파일 bids 형식으로 만들기
import os
import re
import shutil

# 원본 루트 폴더
root_dir = "/nas/research/03-Neural_decoding/3-bids/derivatives/b4_roi"
# 새로운 저장 루트 (BIDS 형식으로 저장할 곳)
output_dir = "/nas/research/03-Neural_decoding/3-bids/derivatives/b4_roi2"

os.makedirs(output_dir, exist_ok=True)

for subject_folder in os.listdir(root_dir):
    subject_path = os.path.join(root_dir, subject_folder)
    if os.path.isdir(subject_path) and subject_folder.startswith("sub-"):
        subject_id = subject_folder.split("-")[1]

        for filename in os.listdir(subject_path):
            if filename.startswith("betas_session") and filename.endswith(".nii.gz"):
                session_match = re.search(r'session(\d+)', filename)
                if session_match:
                    session_num = session_match.group(1).zfill(2)
                    desc = "betaroi"  # 파일 이름
                    new_filename = f"sub-{subject_id}_ses-{session_num}_desc-{desc}.nii.gz"

                    bids_sub_dir = os.path.join(output_dir, f"sub-{subject_id}", f"ses-{session_num}", "func")
                    os.makedirs(bids_sub_dir, exist_ok=True)

                    src_file = os.path.join(subject_path, filename)
                    dst_file = os.path.join(bids_sub_dir, new_filename)
                    shutil.copy2(src_file, dst_file)

In [ ]:
#_rest_ -> _task-rest_ 로 파일들 이름 바꾸기
import os

def rename_rest_to_task_rest(root_dir):
    """
    주어진 root_dir 안의 모든 폴더를 순회하면서
    파일명에 '_rest_'가 포함된 경우 '_task-rest_'로 변경합니다.
    """
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if "_rest_" in filename:
                old_path = os.path.join(dirpath, filename)
                new_filename = filename.replace("_rest_", "_task-rest_")
                new_path = os.path.join(dirpath, new_filename)
                
                # 이름 변경
                os.rename(old_path, new_path)
                print(f"Renamed:\n  {old_path}\n→ {new_path}\n")

if __name__ == "__main__":
    # 여기다가 최상위 폴더 경로 입력
    root_directory = "/nas/research/03-Neural_decoding/3-bids/raw_data/sub-08"  # <-- 여기를 네 데이터 폴더로 바꿔
    rename_rest_to_task_rest(root_directory)


In [1]:
import os
import json

# 비교 대상이 되는 root directory 경로
root_dir = '/nas/research/03-Neural_decoding/3-bids/raw_data'

# 대상 subject
subject = 'sub-06'

# 비교에 사용할 주요 파라미터 키
keys_to_compare = ['RepetitionTime', 'EchoTime', 'SliceTiming', 'PixelSpacing', 'PhaseEncodingDirection']

# 결과 저장용
params_by_file = {}

# 파일 탐색 및 파라미터 수집
for dirpath, dirnames, filenames in os.walk(os.path.join(root_dir, subject)):
    for fname in filenames:
        if fname.endswith('_bold.json'):
            fpath = os.path.join(dirpath, fname)
            with open(fpath, 'r') as f:
                metadata = json.load(f)
                key_info = {k: metadata.get(k, 'MISSING') for k in keys_to_compare}
                params_by_file[fpath] = key_info

# 비교 결과 출력
print(f"\n[{subject}] BOLD JSON 파라미터 비교 결과:")
for fpath, param in sorted(params_by_file.items()):
    print(f"\n▶ {fpath}")
    for k, v in param.items():
        print(f"  - {k}: {v}")



[sub-06] BOLD JSON 파라미터 비교 결과:

▶ /nas/research/03-Neural_decoding/3-bids/raw_data/sub-06/ses-01/func/sub-06_ses-01_task-image_run-01_bold.json
  - RepetitionTime: 1.6
  - EchoTime: 0.022
  - SliceTiming: [0, 0.745, 1.49, 0.63, 1.375, 0.515, 1.2625, 0.4, 1.1475, 0.285, 1.0325, 0.1725, 0.9175, 0.0575, 0.8025, 1.5475, 0.6875, 1.4325, 0.5725, 1.3175, 0.4575, 1.205, 0.3425, 1.09, 0.23, 0.975, 0.115, 0.86, 0, 0.745, 1.49, 0.63, 1.375, 0.515, 1.2625, 0.4, 1.1475, 0.285, 1.0325, 0.1725, 0.9175, 0.0575, 0.8025, 1.5475, 0.6875, 1.4325, 0.5725, 1.3175, 0.4575, 1.205, 0.3425, 1.09, 0.23, 0.975, 0.115, 0.86, 0, 0.745, 1.49, 0.63, 1.375, 0.515, 1.2625, 0.4, 1.1475, 0.285, 1.0325, 0.1725, 0.9175, 0.0575, 0.8025, 1.5475, 0.6875, 1.4325, 0.5725, 1.3175, 0.4575, 1.205, 0.3425, 1.09, 0.23, 0.975, 0.115, 0.86]
  - PixelSpacing: MISSING
  - PhaseEncodingDirection: j-

▶ /nas/research/03-Neural_decoding/3-bids/raw_data/sub-06/ses-01/func/sub-06_ses-01_task-image_run-02_bold.json
  - RepetitionTime: 1.6
  

In [1]:
import shutil
import os

def copy_folder(src, dst):
    # 대상 경로가 이미 존재하면 에러가 발생하므로 먼저 확인
    if os.path.exists(dst):
        print(f"대상 폴더 '{dst}'가 이미 존재합니다.")
        return

    try:
        shutil.copytree(src, dst)
        print(f"'{src}' 폴더를 '{dst}'로 복제했습니다.")
    except Exception as e:
        print(f"복제 중 오류 발생: {e}")

# 예시
src_folder = "/nas/research/03-Neural_decoding/3-bids/raw_data/sub-01"
dst_folder = "/nas/research/03-Neural_decoding/3-bids/derivatives/test"
copy_folder(src_folder, dst_folder)


대상 폴더 '/nas/research/03-Neural_decoding/3-bids/derivatives/test'가 이미 존재합니다.
